In [1]:
import keras.callbacks
import tensorflow as tf
import numpy as np
from PIL import Image
import os
import random
import pandas as pd
from keras import backend as K
import matplotlib.pyplot as plt

## Todo:
1) use tf.data.Dataset to read from type dataset
2) use a dataset fn to apply a transformation such that it randomly selects an m no of images from n images of that pokemon 

In [2]:
class PokemonTypeDataset(tf.data.Dataset):
    @staticmethod
    def one_hot_encode_labels(labels,all_labels:[]):
        label_map={pokemon_type:one_hot_index for one_hot_index, pokemon_type in enumerate(all_labels)} #enum[(0,type1),(1,type2)...]
        one_hot_labels=np.zeros((len(labels),len(all_labels)),dtype=np.int8)
        for index,pokemon_type  in enumerate(labels):
            one_hot_label_index=label_map[pokemon_type]
            one_hot_labels[index,one_hot_label_index]=1
        result_labels = np.sum(one_hot_labels, axis=0)
        return result_labels.astype(np.int8)

    @staticmethod
    def super_generator(all_labels,input_shape:tuple):
        def _generator(csv_dir):
            pokemon_df=pd.read_csv(csv_dir.decode())
            for index,pokemon in pokemon_df.iterrows():
                pokemon_path=pokemon["image_path"]
                if pokemon_path.lower().endswith((".jpg", ".jpeg", ".png")):
                    with Image.open(pokemon_path) as img:
                        if img.mode != 'RGB':
                            img = img.convert('RGB')
                        pokemon_tensor = np.asarray(img.resize((input_shape[0],input_shape[1])))
                        pokemon_tensor = pokemon_tensor / 255.0
                    pokemon_types = []
                    if pd.isna(pokemon["type2"]):
                        pokemon_types.append(pokemon["type1"])
                    else:
                        pokemon_types.extend([pokemon["type1"], pokemon["type2"]])

                    yield pokemon_tensor, PokemonTypeDataset.one_hot_encode_labels(pokemon_types, all_labels)
        return _generator

    def __new__(cls, csv_dir:str,all_labels:list,input_shape:tuple):
        return tf.data.Dataset.from_generator(
            cls.super_generator(all_labels,input_shape),
            args=(csv_dir,),
            output_signature=(
                tf.TensorSpec(shape=input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(None,),dtype=tf.float32)
            )
        ).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [9]:

class PokemonMiniData(tf.data.Dataset):
    @staticmethod
    def one_hot_encode_labels(labels,all_labels:[]):
        label_map={pokemon_type:one_hot_index for one_hot_index, pokemon_type in enumerate(all_labels)} #enum[(0,type1),(1,type2)...]
        one_hot_labels=np.zeros((len(labels),len(all_labels)),dtype=np.int8)
        for index,pokemon_type  in enumerate(labels):
            one_hot_label_index=label_map[pokemon_type]
            one_hot_labels[index,one_hot_label_index]=1
        result_labels = np.sum(one_hot_labels, axis=0)
        #shape->(len(labels),len(all_labels))
        return result_labels.astype(np.int8)
    
    @staticmethod
    def super_generator(all_labels,input_shape:tuple):
        def _generator(csv_dir):
            pokemon_df=pd.read_csv(csv_dir.decode())
            pokemon_df.head()
            # pokemon_df=pokemon_df[pokemon_df["type1"]=="Grass"]
            for index,pokemon in pokemon_df.iterrows():
                if pokemon_df["type1"].iloc[0]=="Grass":
                    pokemon_path=pokemon["image_path"]
                    if pokemon_path.lower().endswith((".jpg", ".jpeg", ".png")):
                        with Image.open(pokemon_path) as img:
                            if img.mode != 'RGB':
                                img = img.convert('RGB')
                            pokemon_tensor = np.asarray(img.resize((input_shape[0],input_shape[1])))
                            pokemon_tensor = pokemon_tensor / 255.0
                        pokemon_types = []
                        if pd.isna(pokemon["type2"]):
                            pokemon_types.append(pokemon["type1"])
                        else:
                            pokemon_types.extend([pokemon["type1"], pokemon["type2"]])

                        yield pokemon_tensor, PokemonTypeDataset.one_hot_encode_labels(pokemon_types, all_labels)
        return _generator

    def __new__(cls, csv_dir:str,all_labels:list,input_shape:tuple):
        return tf.data.Dataset.from_generator(
            cls.super_generator(all_labels,input_shape),
            args=(csv_dir,),
            output_signature=(
                tf.TensorSpec(shape=input_shape, dtype=tf.float32),
                tf.TensorSpec(shape=(None,),dtype=tf.float32)
            )
        ).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [10]:
dataset_dir="../PokemonData"
csv_dir="../csv/FirstGenPokemonCSVDataset.csv"
batch_size=4
pokemon_df=pd.read_csv(csv_dir)

all_labels_1=pokemon_df["type1"].unique()
all_labels_2=pokemon_df["type2"].unique()
all_labels = list(set(all_labels_1) | set(all_labels_2) - {np.NaN})
input_shape=(256,256,3)
# grass_df=pokemon_df[pokemon_df["type1"]=='Grass']
# all_labels_1=grass_df["type1"].unique()
# all_labels_2=grass_df["type2"].unique()
# all_labels=list(set(all_labels_1) | set(all_labels_2)- {np.NaN})
all_labels


['ice',
 'psychic',
 'water',
 'electric',
 'rock',
 'fire',
 'flying',
 'normal',
 'poison',
 'fighting',
 'ghost',
 'ground',
 'dragon',
 'bug',
 'grass']

In [14]:
all_labels,len(all_labels)

(['ice',
  'psychic',
  'water',
  'electric',
  'rock',
  'fire',
  'flying',
  'normal',
  'poison',
  'fighting',
  'ghost',
  'ground',
  'dragon',
  'bug',
  'grass'],
 15)

In [15]:
dataset=PokemonMiniData(csv_dir,all_labels,input_shape)
dataset=dataset.batch(batch_size,num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat(4)

In [16]:
dataset_size=0
for element in dataset.as_numpy_iterator():
    
    
    dataset_size+=1

In [8]:
dataset_size=dataset_size
print(f"dataset size: {dataset_size}")

dataset size: 0


In [9]:
'''
Test and training data
'''
shuffled_dataset=dataset.shuffle(buffer_size=dataset_size,seed=69) 

train_size=int(0.8*dataset_size)
test_size=dataset_size-train_size
train_ds=shuffled_dataset.take(train_size)
test_ds=shuffled_dataset.skip(train_size)

In [10]:
pokemon_types_len=len(all_labels)

In [11]:
pokemon_types_len

18

In [12]:
steps_per_epoch=tf.math.ceil(dataset_size/batch_size)
validation_steps=tf.math.ceil(test_size/batch_size)
steps_per_epoch,validation_steps

(<tf.Tensor: shape=(), dtype=float32, numpy=514.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=103.0>)

In [23]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy')>=0.99:
            print('Test accuracy: %.2f%%' % (logs.get('accuracy')*100))
            self.model.stop_training = True

            

In [25]:
def type_model(num_classes,input_shape):
    resnet_model=tf.keras.applications.ResNet50(include_top=False,input_shape=input_shape)
    for layer in resnet_model.layers:
        layer.trainable=False
    
    flatten_1=tf.keras.layers.Flatten()(resnet_model)
    dense_1=tf.keras.layers.Dense(512, activation='relu')(flatten_1)
    
    output=tf.keras.layers.Dense(num_classes,activation='sigmoid')(dense_1)
    
    model=tf.keras.models.Model(inputs=resnet_model.inputs,outputs=output)
    
    return model


In [26]:
my_callback=MyCallback()

In [30]:
model=type_model(pokemon_types_len,input_shape=input_shape)
model.compile(optimizer="Adam",loss=tf.keras.losses.BinaryCrossentropy(),metrics=["accuracy"])

ImportError: `load_weights` requires h5py package when loading weights from HDF5. Try installing h5py.

In [34]:
import h5py

In [142]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2_1 (Conv2D)            (None, 252, 252, 16)         1216      ['input_11[0][0]']            
                                                                                                  
 conv1 (Conv2D)              (None, 252, 252, 16)         1216      ['input_11[0][0]']            
                                                                                                  
 pool2_1 (MaxPooling2D)      (None, 84, 84, 16)           0         ['conv2_1[0][0]']             
                                                                                            

In [143]:
history=model.fit(x=dataset,epochs=10,batch_size=batch_size,steps_per_epoch=steps_per_epoch)

Epoch 1/10
514/514 [==============================] - 89s 169ms/step - loss: 0.3025 - accuracy: 0.0648
Epoch 2/10
514/514 [==============================] - 86s 166ms/step - loss: 0.2853 - accuracy: 0.0823
Epoch 3/10
514/514 [==============================] - 93s 182ms/step - loss: 0.2819 - accuracy: 0.0857
Epoch 4/10
514/514 [==============================] - 87s 170ms/step - loss: 0.2788 - accuracy: 0.1022
Epoch 5/10
514/514 [==============================] - 0s 36us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


In [146]:
def plot_filtered_pokemon(tensor:tf.Tensor,model:tf.keras.Model):
    get_conv=K.function(
        [model.layers[0].input],
    )
    filtered_tensor=get_conv(tensor)
    print(filtered_tensor[0].shape)
    return filtered_tensor

In [169]:
pikachu=f"{dataset_dir}/Chandelure/1.jpg"
pikachu_tensor=tf.keras.preprocessing.image.load_img(pikachu).resize((input_shape[0],input_shape[1]))
pikachu_tensor=tf.keras.preprocessing.image.img_to_array(pikachu_tensor)
pikachu_tensor=np.expand_dims(pikachu_tensor,axis=0)
pikachu_tensor.shape

(1, 256, 256, 3)

In [170]:
prediction=model.predict(pikachu_tensor)
print(prediction)

1/1 [==============================] - 0s 36ms/step
[[9.9985349e-01 8.6366653e-24 6.3556202e-13 1.0000000e+00 1.0000000e+00
  3.6680303e-05 9.9896955e-01 1.5084625e-11 8.7832323e-06 1.0000000e+00
  1.4783147e-24 4.0116979e-13 1.3912978e-14 1.0000000e+00 9.9987167e-01
  1.0000000e+00 1.0000000e+00 2.0856613e-05]]


In [172]:
def plot_filtered_pokemon(tensor, model):
    get_conv = tf.keras.backend.function(
        [model.layers[0].input],
        [model.layers[0].output]
    )

    filtered_tensor = get_conv(tensor)[0]

    num_features = filtered_tensor.shape[-1]
    nrows=2
    ncols=2
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(256, 256))
    feature=0
    for x in range(nrows):
        for y in range(ncols):
            ax[x,y].imshow(filtered_tensor[0,:,:,feature])
            feature+=1

    return filtered_tensor
filtered_tensor = plot_filtered_pokemon(pikachu_tensor, model)
filtered_tensor.shape

(1, 252, 252, 32)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("kvpratama/pokemon-images-dataset")

In [21]:
type(dataset)

datasets.dataset_dict.DatasetDict